# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Defaulting to user installation because normal site-packages is not writeable


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [14]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [15]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [16]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
]

In [17]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url="https://api.xiaoai.plus/v1")

In [18]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [19]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [20]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [21]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [22]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题']


In [11]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题


In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [13]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the

In [23]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [24]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': 'What is blackbody radiation?'}
> Finished chain.
{'input': 'What is blackbody radiation?', 'text': '\n\n黑体辐射是指处于热平衡状态的物体所发出的电磁辐射，其频率和强度与物体的温度有关。根据普朗克定律，黑体辐射的频谱分布与温度有关，并且在某一特定温度下，最大辐射强度出现在一个特定的频率上，称为峰值频率。黑体辐射是一种重要的现象，它在理论物理中扮演着重要的角色，并且也有许多实际应用，例如太阳辐射和热辐射。希望这可以回答您的问题。如果您有任何其他问题，请随时提出。'}


In [25]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': 'What is the first prime number greater than 40, such that adding one to this prime number is divisible by 3?'}
> Finished chain.
{'input': 'What is the first prime number greater than 40, such that adding one to this prime number is divisible by 3?', 'text': '\n\n首先，我们需要找到大于40的第一个质数。质数是只能被1和自身整除的数，因此我们可以从41开始依次往上找质数。经过验证，41是大于40的第一个质数。\n\n接着，我们需要找到一个数，使得41加上它之后能被3整除。因为如果一个数能被3整除，那么它的余数只能是0、1或2，因此我们可以考虑从0、1、2开始依次加上41，直到能够被3整除的数。经过计算，我们发现41加上1后能被3整除。\n\n因此，答案为42，即大于40的第一个质数，使得加上1后能被3整除。'}


In [26]:
router_chain.verbose = True

In [27]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种天体，它的引力非常强大，甚至连光都无法逃离它的吸引力。它的形成是由于质量非常大的恒星坍塌而成，形成了极其密集的物质。黑洞非常神秘，因为我们无法直接观测它，只能通过它对周围物质的影响来推测它的存在。关于黑洞的研究仍在进行中，我们仍然需要更多的科学实验和研究来更好地了解它。我不知道所有关于黑洞的答案，但我会尽力回答您的问题。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [33]:
bio_template = """你是一位知识渊博的生物学家。
你能用清晰的解释回答有关生物学的问题。
如果遇到你不确定的问题，你会诚实地说出来。

这是一个问题：
{input}"""

cs_template = """你是一位经验丰富的计算机科学专家。
你能够回答各种计算机科学和技术的问题。
遇到不明确的问题时，你会寻求更多信息。

这是一个问题：
{input}"""

chinese_lit_template = """你是一位对汉语言文学有深刻理解的教授。
你可以解答关于古典文学、现代文学以及语言学的疑问。
如果一个问题超出了你的专长范围，你会开诚布公地说明。

这是一个问题：
{input}"""

english_template = """你是一位精通英语的老师。
你可以回答关于英语语法、词汇和文学的问题。
假如有些问题超出了你的知识范围，你会诚实地表明。

这是一个问题：
{input}"""

In [34]:

bio_prompt = PromptTemplate(template=bio_template, input_variables=["input"])


cs_prompt = PromptTemplate(template=cs_template, input_variables=["input"])


chinese_lit_prompt = PromptTemplate(template=chinese_lit_template, input_variables=["input"])


english_prompt = PromptTemplate(template=english_template, input_variables=["input"])


bio_chain = LLMChain(llm=llm, prompt=bio_prompt)
cs_chain = LLMChain(llm=llm, prompt=cs_prompt)
chinese_lit_chain = LLMChain(llm=llm, prompt=chinese_lit_prompt)
english_chain = LLMChain(llm=llm, prompt=english_prompt)


destination_chains["生物"] = bio_chain
destination_chains["计算机"] = cs_chain
destination_chains["汉语言文学"] = chinese_lit_chain
destination_chains["英语"] = english_chain

In [35]:
prompt_infos.extend([
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": bio_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机科学问题",
        "prompt_template": cs_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_lit_template,
    },
    {
    "name": "英语",
    "description": "适用于回答英语问题",
    "prompt_template": english_template,
    }
])

In [36]:

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)


chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [37]:

bio_prompt = PromptTemplate(template=bio_template, input_variables=["input"])
cs_prompt = PromptTemplate(template=cs_template, input_variables=["input"])
chinese_lit_prompt = PromptTemplate(template=chinese_lit_template, input_variables=["input"])
english_lit_prompt = PromptTemplate(template=english_template, input_variables=["input"])


bio_chain = LLMChain(llm=llm, prompt=bio_prompt)
cs_chain = LLMChain(llm=llm, prompt=cs_prompt)
chinese_lit_chain = LLMChain(llm=llm, prompt=chinese_lit_prompt)
english_lit_chain = LLMChain(llm=llm, prompt=english_lit_prompt)


destination_chains["生物"] = bio_chain
destination_chains["计算机"] = cs_chain
destination_chains["汉语言文学"] = chinese_lit_chain
destination_chains["英语"] = english_lit_chain


destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)






In [38]:
test_input = "解释一下生物学中的光合作用。"
response = chain.run(input=test_input)
print(response)  



> Entering new MultiPromptChain chain...
生物: {'input': '解释一下生物学中的光合作用。'}
> Finished chain.


光合作用是指植物和其他光合有机生物利用太阳能将二氧化碳和水转化为有机物质和氧气的过程。它是生物界最重要的化学反应之一，也是地球上所有生物生存的基础。在光合作用中，光能被光合色素吸收，通过光合作用反应中的一系列化学反应，最终将二氧化碳和水转化为葡萄糖等有机物，同时释放出氧气。这些有机物可以被植物和其他生物作为能量和营养物质来源。光合作用不仅为生物提供了能量和营养，还通过消耗二氧化碳和产生氧气来维持地球上的气候平衡。


In [39]:
test_input = "什么是面向对象编程中的继承？"
response = chain.run(input=test_input)
print(response)  



> Entering new MultiPromptChain chain...
计算机: {'input': '什么是面向对象编程中的继承?'}
> Finished chain.


面向对象编程中的继承是指一个类（也称为子类）可以继承另一个类（也称为父类）的属性和方法。子类可以使用父类中已有的属性和方法，并且还可以定义自己独有的属性和方法。这种继承关系可以让程序更加灵活和可扩展，同时也提高了代码的重用性。


In [40]:
test_input = "古诗《静夜思》表达了诗人怎样的情感？"
response = chain.run(input=test_input)
print(response)  



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '古诗《静夜思》表达了诗人怎样的情感？'}
> Finished chain.


古诗《静夜思》是唐代诗人李白所作，其表达的情感是对于孤独和思乡的感受。诗中描绘了一位在夜晚独自思念故乡的诗人，通过对月亮、床、窗等物象的描写，表达了他内心深处的孤独和对故乡的思念。整首诗朴素而又富有意境，给读者留下了深刻的印象。


In [42]:
test_input = "What's the difference between 'affect' and 'effect' in English?"
response = chain.run(input=test_input)
print(response)  



> Entering new MultiPromptChain chain...
英语: {'input': "What's the difference between 'affect' and 'effect' in English?"}
> Finished chain.


这是一个常见的问题，许多学生都会遇到困惑。实际上，这两个词在拼写和发音上很相似，但它们有不同的含义。'Affect'是一个动词，意思是影响或改变某物；而'effect'可以是一个名词或动词，意思是结果、影响或产生。例如，"The weather affected our plans for the day."（天气影响了我们的一天计划。）"The medicine had a positive effect on her health."（这种药对她的健康有积极的影响。）因此，简单来说，'affect'表示影响，'effect'表示结果或影响的效果。希望这能帮助你理解这两个词的区别。
